# Assignemt XX

## Imports

In [64]:
import kagglehub
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

## Download

In [ ]:
# Download latest version
path = kagglehub.dataset_download("abhi8923shriv/sentiment-analysis-dataset")

print("Path to dataset files:", path)

100%|██████████| 54.4M/54.4M [02:00<00:00, 474kB/s] 

Extracting files...


Path to dataset files: C:\Users\abarhouche\.cache\kagglehub\datasets\abhi8923shriv\sentiment-analysis-dataset\versions\9


## Read + EDA

In [62]:
train_df = pd.read_csv("../Data/train_data.csv", encoding="latin1")
cols = [x.split('\xa0')[0] for x in list(train_df.columns)]
test_df = pd.read_csv("../Data/test_data.csv", header=None, names=cols)
train_df.columns = cols

train = pd.read_csv('../Data/train.csv', encoding="latin1")

- train_df is missing a category 2 
- concat train with train df to add the category after transformation
- eda accordingly

(same applies for test)

 - check length dist of the texts (#words)
 - check unique words, check unique lemmas accros the whole corpus...


In [61]:
train.columns

Index(['textID', 'text', 'selected_text', 'sentiment', 'Time of Tweet',
       'Age of User', 'Country', 'Population -2020', 'Land Area (Km²)',
       'Density (P/Km²)'],
      dtype='object')

,id of the tweet,sentiment,polarity of tweet


In [45]:
train_df['id of the tweet'].nunique()

1048041

In [38]:
cols

['polarity of tweet',
 'id of the tweet',
 'date of the tweet',
 'query',
 'user',
 'text of the tweet']

In [39]:
train_df.columns

Index(['polarity of tweet', 'id of the tweet', 'date of the tweet', 'query',
       'user', 'text of the tweet'],
      dtype='object')

In [40]:
train_df = train_df[["text of the tweet", "polarity of tweet"]].drop_duplicates(keep = "first")
test_df = test_df[["text of the tweet", "polarity of tweet"]].drop_duplicates(keep = "first")

# cols

In [41]:
train_df['polarity of tweet'].unique()

array([0, 4], dtype=int64)

In [42]:
test_df['polarity of tweet'].unique()

array([4, 0, 2], dtype=int64)

## TF-IDF

In [65]:
tfidf_ngram_vectorizer = TfidfVectorizer(min_df=0.001, ngram_range=(1, 3))
tfidf_ngram_vectorizer_train = tfidf_ngram_vectorizer.fit_transform(train_df["text of the tweet"])
tfidf_ngram_vectorizer.get_feature_names_out()[150:160]
# tfidf_ngram_vectorizer_val = tfidf_ngram_vectorizer.transform(validation['user_review'])


array(['back to the', 'back to work', 'bad', 'bad for', 'badly', 'bag',
       'band', 'bank', 'bar', 'bb'], dtype=object)

In [ ]:
dense_vector = tfidf_ngram_vectorizer_train[0].toarray()
dense_vector.shape
# input should be 2094
# last layer 3

(1, 2094)

In [75]:
import numpy as np
non_zero_indices = np.where(dense_vector[0] != 0)[0]
non_zero_values = dense_vector[0][non_zero_indices]
non_zero_values

array([0.25332477, 0.11494955, 0.20549252, 0.32523437, 0.21458021,
       0.15907052, 0.27399198, 0.28517186, 0.20122084, 0.23521897,
       0.12217779, 0.11609644, 0.30733876, 0.25530838, 0.21868964,
       0.13879416, 0.17106519, 0.28526882, 0.29201414])

In [ ]:
# converting the features_train and features_validation 
features_train = torch.tensor(features_train.toarray(), dtype=torch.float32)
features_validation = torch.tensor(features_validation.toarray(), dtype=torch.float32)

## Model + Training + Eval

In [ ]:
# convert target variables into pytorch tensors
y_train = torch.tensor(train['user_suggestion'])
y_validation = torch.tensor(validation['user_suggestion'])

In [ ]:
# initialise TensorDataset  object
train_dataset = TensorDataset(features_train, y_train)
val_dataset = TensorDataset(features_validation, y_validation)

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=True)

In [ ]:
class ANNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_rate):
        super(ANNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.fc3 = nn.Sigmoid() #softmax

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

In [ ]:
model = ANNModel(input_size=features_train.shape[1], hidden_size=64, output_size=1, dropout_rate=0.5)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [ ]:
import matplotlib.pyplot as plt

num_epochs = 30  # Number of epochs
losses = []  # List to store the average loss per epoch
val_losses = []

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    total_loss = 0  # Variable to store the total loss in each epoch
    total_val_loss = 0
    count = 0  # Variable to count the number of batches
    val_count = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        outputs = outputs.squeeze()  # Squeeze the output to match the label's shape
        loss = criterion(outputs, labels.float())  # Ensure labels are float
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        count += 1
    average_loss = total_loss / count  # Calculate average loss for the epoch
    losses.append(average_loss)  # Append average loss to the list
       
    for inputs, labels in val_loader:
        val_outputs = model(inputs)
        val_outputs = val_outputs.squeeze()  # Squeeze the output to match the label's shape
        val_loss = criterion(val_outputs, labels.float())  # Ensure labels are float
        total_val_loss += val_loss.item()
        val_count += 1
    average_val_loss = total_val_loss / val_count  # Calculate average loss for the epoch
    val_losses.append(average_val_loss)  # Append average loss to the list
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {average_loss:.4f}, Val Loss: {average_val_loss:.4f}')

# Plotting the training loss
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs + 1), losses, marker='o', linestyle='-', color='b', label = 'train_loss')
plt.plot(range(1, num_epochs + 1), val_losses, marker='o', linestyle='-', color='r', label = 'val_loss')
plt.title('Training Loss per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# copy from module 4 of course 05

In [ ]:
def calculate_accuracy(loader):
    model.eval()  # Set the model to evaluation mode
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in loader:
            outputs = model(inputs)
            predicted = outputs.squeeze() > 0.5  # Apply threshold to convert probabilities to binary predictions
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

train_accuracy = calculate_accuracy(train_loader)
val_accuracy = calculate_accuracy(val_loader)

print(f'Training Accuracy: {train_accuracy}%')
print(f'Validation Accuracy: {val_accuracy}%')